Steps done for this task:

1. Copy and paste short video for each category (push, other) manually in separate folder (each video contains either pushing or other activity--> this method made labeling automatically)
2. Rename the videos based on their labeling. e.g. : push_01 , push_02, .., and  other_01, other_02, ..
3. manually pasted all the renamed videos in one directory called video_dir
4. Capture videos from video_dir and save the captured frames into image_dir
5. configure the model 
*******************
6. Since frames names start with either push or other, labels are assigned as 1 or zero automatically

### Note: This model is traied on Google Colab  with more number of data ( explained in report)

In [3]:
import cv2     # for capturing videos
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
from keras.utils import to_categorical
import numpy as np    
import os
import glob
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

Using TensorFlow backend.


In [1]:
# I copied and pasted the raw videos manually in their corresponding folders.Then rename them corresponding to
# their labels (for example: push_1 to push_100 and other_1 to other_100)

In [ ]:
# This code need to be run only one time  

def rename_file (path , name):
    i=0                                         
    for filename in os.listdir(path):
        os.rename(os.path.join(path,filename), os.path.join(path, name + str(i)+'.avi'))
        i = i +1

all_video_push = "D:\\tamu\\courses\\DeepLearning\\ProjectPart4\\pushing"
all_video_other = "D:\\tamu\\courses\\DeepLearning\\ProjectPart4\\other"
        
rename_file(all_video_push, 'push_')
rename_file(all_video_other, 'other_')

In [3]:
# after renaming all videos (push and other) are manually pasted in one directory --> video_dir
video_dir = "D:\\tamu\\courses\\DeepLearning\\ProjectPart4\\all_video"
img_dir = "D:\\tamu\\courses\\DeepLearning\\ProjectPart4\\all_img"

## capture video

In [8]:
def get_frames(video_path, dest_path):
    file_name = video_path.split('.')[0].split('\\')[-1]
    vidcap = cv2.VideoCapture(video_path)

    sec = 0
    frameRate = 0.5           #//it will capture image in each 0.5 second
    count = 1
    hasFrames = True
    while hasFrames:
        vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
        hasFrames,image = vidcap.read()
        if hasFrames:
            cv2.imwrite(dest_path+'\\'+file_name+'_'+ str(count)+".jpg", image)     # save frame as JPG file

        count = count + 1
        sec = sec + frameRate
        sec = round(sec, 2)

In [ ]:
# getting captions from videos and put them in related derectories:

for i in range(len(os.listdir(video_dir))):
    video_path = video_dir + '\\' + os.listdir(video_dir)[i]
    dest_path = img_dir

    get_frames(video_path, dest_path)

### Getting labels
assign 0 to the images labeled as 'other' and assign 1 to the images labeld as 'push'

In [55]:
train_label =[]
for file in os.listdir(img_dir):
    label = file.split('_')[0]
    if label == 'other':
        train_label.append(0)
    else:
        train_label.append(1)


## Note
the number of video frmaes need to be the same. For this submission I manually handeled it to get equal number of frames for each video.
Later I will use padding and masking layer

### Image preprocessing
-resizing all images to (250,250)

In [51]:
def get_Xtrain(file_name):
    X_train = []
    for i , filename in enumerate(glob.glob(file_name)):
        image = cv2.imread(filename)
        res_img = cv2.resize(image, dsize=(250,250), interpolation=cv2.INTER_CUBIC)
        X_train.append(res_img)
    
    return X_train 

X = get_Xtrain(img_dir + '\\*.jpg')

In [56]:
# checking the size of the resized images (X)
X[0].shape
print(len(X))

252


## Get numpy array of images

In [57]:
#transfering the train_list and label_list to array
x_train = np.asarray(X)
y_train = np.asarray(train_label)

In [58]:
print('x_train.shape: ', x_train.shape)
print('y_train.shape: ', y_train.shape)

x_train.shape:  (252, 250, 250, 3)
y_train.shape:  (252,)


In [59]:
## Normalizing X dataset:
x_train = x_train/x_train.max()                  

In [60]:
# Chaning the shape of the input for TimeDistributed layer in the model
x = []
y = []
for i in range(0, x_train.shape[0], 10):
    x.append(x_train[i:(i+10)])
    y.append(y_train[i:(i+10)])

x = np.asarray(x)
y = np.asarray(y)
print(x.shape)
print(y.shape)

x_train = x
y_train = y

(24, 10, 250, 250, 3)
(24, 10)


In [61]:
y_train = y_train.reshape(y_train.shape[0], 10,1)
print(y_train.shape)

(24, 10, 1)


## Model Configuration
Not using transfer learning

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, Dropout,Flatten, MaxPooling2D, BatchNormalization, TimeDistributed
from keras.layers import LSTM, Dense


n_timesteps = 10                                         #  ==> nums of frames for each video (question above)
lstm_cells =5

model = Sequential()
model.add(TimeDistributed(Conv2D (16, (3,3), activation = 'relu'),
                                  input_shape = (n_timesteps, 250,250,3)))      
model.add(TimeDistributed(Conv2D (32, (3,3), activation = 'relu')))             
model.add(TimeDistributed(Dropout(0.5)))
model.add(TimeDistributed(MaxPooling2D((2,2))))        
#model.add(TimeDistributed(Conv2D(64, (3,3), activation = 'relu')))             
#model.add(TimeDistributed(Conv2D(256, (3,3), activation = 'relu')))
model.add(TimeDistributed(Dropout(0.5)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2,2))))
#model.add(TimeDistributed(Conv2D (512, (3,3), activation = 'relu')))
#model.add(TimeDistributed(MaxPooling2D((2,2))))
#model.add(TimeDistributed(Conv2D (512, (3,3), activation = 'relu')))
#model.add(TimeDistributed(MaxPooling2D((2,2))))
#model.add(TimeDistributed(BatchNormalization(momentum = momentum)))
#model.add(TimeDistributed(Dropout(0.5)))
model.add(TimeDistributed(Flatten()))                                                      #4
#model.summary()


#model.add(LSTM(batch_size = n_batches, timesteps = n_timesteps, input_dim = (n_features,)))
model.add(LSTM(units = lstm_cells, return_sequences = True, dropout=0.1, recurrent_dropout=0.5))
#model.add(Dense(512, activation='relu'))     # you may need more Dense layers
#model.add(Dense(256, activation='relu'))
#model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()


#1: use conv2d to extract image features + wrap the subsequences through TimeDistributed
#2: Dropout layer to overcome overfitting
#3: MaxPooling the grab the most representative features from each featuremap
#4: Flatten the tensors so as to be ready for LSTM layer
#6: Final layer to predict the intended action (Binary output)
## Note: Check if using pre-trained model outperforms 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_6 (TimeDist (None, 10, 248, 248, 16)  448       
_________________________________________________________________
time_distributed_7 (TimeDist (None, 10, 246, 246, 32)  4640      
_________________________________________________________________
time_distributed_8 (TimeDist (None, 10, 246, 246, 32)  0         
_________________________________________________________________
time_distributed_9 (TimeDist (None, 10, 123, 123, 32)  0         
_________________________________________________________________
time_distributed_10 (TimeDis (None, 10, 123, 123, 32)  0         
_________________________________________________________________
time_distributed_11 (TimeDis (None, 10, 123, 123, 32)  128       
_________________________________________________________________
time_distributed_12 (TimeDis (None, 10, 61, 61, 32)   

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(lr=2e-5)
              metrics=['accuracy']) 

# optimizer='rmsprop'

In [ ]:
epochs = 20
history = model.fit(x=x_train, y=y_train, validation_split = 0.2, batch_size=10, epochs=epochs, shuffle=True)

Train on 19 samples, validate on 5 samples
Epoch 1/3
